# 0. Imports

In [1]:
from h2ogpte import H2OGPTE
import numpy as np
import pandas as pd
import os

# 1. Data Sets

In [3]:
songs = pd.read_csv('../../data/songs_short.csv')
user = pd.read_csv('../../data/user.csv')
history = pd.read_csv('../../data/listening_history.csv')

In [ ]:
print('Songs:')
print(songs.head())

print('Users:')
print(user.head())

print('User listening history:')
print(history.head())

Songs:
               spotify_id                          Song         Artist  Rank  \
0  4H2TRR9FjnnIwxGnIt9stO                  Killin' Time    Clint Black     1   
1  6iy4PoAuZBMvtrlDX4VxC7               Check Yes Or No  George Strait     2   
2  3oW6SWwGqiZSPTiAp7ZQoH  It's A Great Day To Be Alive   Travis Tritt     3   
3  7lUE02KHkZM44BZgjCaWRO            Meet in the Middle    Diamond Rio     4   
4  1ecIqo8pOc8S6y3l80iGxd                         Fancy  Reba McEntire     5   

     Genre  
0  country  
1  country  
2  country  
3  country  
4  country  
Users:
     UserID  Password                 Name Genre_1  Genre_2  Genre_3
0     sibyl  7S02NXS3           Sibyl Bode    rock      r&b      pop
1   eleazar  7S02NXS4    Eleazar Hettinger     rap     soul  hip hop
2  geovanny  7S02NXS5        Geovanny Lang     rap  country     soul
3     micah  7S02NXS6    Micah Satterfield    jazz      r&b     soul
4  courtney  7S02NXS7  Courtney Schowalter    soul     folk    metal
User listenin

# 2. h2o Model

In [21]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-cfLVheKJ5GJMA7oEJoPKXPOul4aWtCk9fnMD0BiYBYjaH53J',
)

In [22]:
# Create Collection
collection_id = client.create_collection(
    name="MusicRecco",
    description="Music Reccommender",
)

In [23]:
# Convert Data to Readable Format
songs_txt = songs.to_csv(index=False)
user_txt = user.to_csv(index=False)
history_txt = history.to_csv(index=False)

In [24]:
# Upload
songs_data = client.upload('songs_short.txt', songs_txt.encode())
user_data = client.upload('user.txt', user_txt.encode())
history_data = client.upload('listening_history.txt', history_txt.encode())

# Ingest the uploaded data
client.ingest_uploads(collection_id, [songs_data, user_data, history_data])

Job(id='782b4a5a-0694-4230-92f6-85033c866317', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 4, 7, 4, 18, 10, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='a3624524b3444461893c715976f10ce6', status='Indexing done.'), JobStatus(id='eb7cc35f5c284ae38b2df4a30a90d0ae', status='Collecting done.')], errors=[], last_update_date=datetime.datetime(2024, 4, 7, 4, 18, 32, tzinfo=TzInfo(UTC)), duration='22s', duration_seconds=22.0)

# 3. ChatBot Page

In [56]:
import re

In [25]:
#Initiate Chat
chat_session_id = client.create_chat_session(collection_id)

In [ ]:
'Recommend less than 5 music unless told otherwise. Return only the song name, unless stated otherwise. Return spotify_id of each recommended music at the end of each recomendation'

In [124]:
def spot_extraction(msg):
    with client.connect(chat_session_id) as session:
        # Simple Question for Document Collection
            answer = session.query(
            message= msg,
            system_prompt ='Recommend not more than 5 music. MUST return spotify_id at the end of each song.',
            rag_config={
                "rag_type": "rag",
            },
            ).content
    return answer

In [147]:
ans = spot_extraction("What are some music that i can listen to when im sad")
print(ans)

Here are five songs that you might find comforting when you're feeling down, based on the information provided in the document:

1. "Yesterday" by The Beatles (page 17, row 1, spotify\_id: "6FRwDxXsvSasw0y2eDArsz")
2. "Imagine" by John Lennon (page 17, row 2, spotify\_id: "1bp2IO61zbQrbWNmKKxg3f")
3. "Hallelujah" by Leonard Cohen (page 17, row 3, spotify\_id: "3qiyyUfYe7CRYLucrPmulD")
4. "Someone Like You" by Adele (page 18, row 1, spotify\_id: "5uvosCdMlFdTXhoazkTI5R")
5. "Fix You" by Coldplay (page 18, row 2, spotify\_id: "6mHOcVtsHLMuesJkswc0GZ")

Please note that these song recommendations are based solely on the data provided in the document and do not take into account personal preferences or musical tastes.


In [148]:
ans1 = spot_extraction("What are some music that you recommend me")
print(ans1)

Based on the data provided, here are five songs from different artists that I recommend:

1. "Light My Fire" by The Doors (spotify\_id: 5uvosCdMlFdTXhoazkTI5R)
2. "The Logical Song - 2010 Remastered" by Supertramp (spotify\_id: 6mHOcVtsHLMuesJkswc0GZ)
3. "Barracuda" by Heart (spotify\_id: 4KfSdst7rW39C0sfhArdrz)
4. "Like a Stone" by Audioslave (spotify\_id: 3YuaBvuZqcwN3CEAyyoaei)
5. "Back In Black" by AC/DC (spotify\_id: 08mG3Y1vljYA6bvDt4Wqkj)

These songs are from well-known rock bands and have high popularity scores according to the data.


In [129]:
ans2 = spot_extraction("music that i can dance to the most. Return the spotify id too")

In [149]:
print(ans2)

Based on the "danceability" feature in the given dataset, here are five songs that have high danceability scores:

1. "Kinetic (The Crystal Method vs. Dada Life)" by The Crystal Method, with a danceability score of 0.616. Spotify ID: 6s0yNmp4Hd32wGx40T6uL8
2. "Galvanize" by The Chemical Brothers, with a danceability score of 0.745. Spotify ID: 4bz7uB4edifWKJXSDxwHcs
3. "God Is a DJ" by Faithless, with a danceability score of 0.606. Spotify ID: 1pUFYb9peWkK8m1WCKNRjp
4. "In Yer Face - Bicep Remix" by 808 State, with a danceability score of 0.616. Spotify ID: 6DxKG8EHEqkWdKrFQSvm32
5. "Teardrop" by Massive Attack, with a danceability score of 0.615. Spotify ID: 67Hna13dNDkZvBpTXRIaOJ

Note that these recommendations are based solely on the danceability metric and do not take into account other factors such as genre, tempo, or personal preference.


In [206]:
def make_link(text):
    lines = text.split('\n')
    lines = [line.strip() for line in lines if line.strip()]
    regex_pattern =  r'\b[A-Za-z0-9]{22}\b'

    # Extracting Spotify IDs
    spotify_ids = []
    for string in lines:
        match = re.findall(regex_pattern, string)
        if match:
            spotify_ids.append(match)
    
    #output clickable links
    for idx, spotify_id in enumerate(spotify_ids, start=1):
        link = f"https://open.spotify.com/track/{spotify_id[0]}"
        print(f"{link}")


In [207]:
make_link(ans)

https://open.spotify.com/track/6FRwDxXsvSasw0y2eDArsz
https://open.spotify.com/track/1bp2IO61zbQrbWNmKKxg3f
https://open.spotify.com/track/3qiyyUfYe7CRYLucrPmulD
https://open.spotify.com/track/5uvosCdMlFdTXhoazkTI5R
https://open.spotify.com/track/6mHOcVtsHLMuesJkswc0GZ


In [167]:
import re

def extract_spotify_ids(strings):
    # Regular expression pattern to match Spotify IDs
    pattern = r"(?:Spotify ID|spotify_id)[:\s]*\"?([a-zA-Z0-9_-]{22})\"?"

    # Extracting Spotify IDs
    spotify_ids = []
    for string in strings:
        match = re.search(pattern, string)
        if match:
            spotify_ids.append(match.group(1))

    return spotify_ids

# Example usage:
strings = [
    "1. \"Yesterday\" by The Beatles (page 17, row 1) - Spotify ID: 6FRwDxXsvSasw0y2eDArsz",
    "2. \"The Sound of Silence\" by Simon & Garfunkel (page 18, row 2) - Spotify ID: 5uvosCdMlFdTXhoazkTI5R",
    "3. \"Imagine\" by John Lennon (page 19, row 1) - Spotify ID: 6mHOcVtsHLMuesJkswc0GZ",
    "4. \"Hallelujah\" by Leonard Cohen (page 20, row 2) - Spotify ID: 0sgDEFq9oeC8ueOK0mR5IA",
    "5. \"Someone Like You\" by Adele (page 20, row 3) - Spotify ID: 4KfSdst7rW39C0sfhArdrz",
    "6. \"Light My Fire\" by The Doors (spotify_id: 5uvosCdMlFdTXhoazkTI5R)",
    "7. \"The Logical Song - 2010 Remastered\" by Supertramp (spotify_id: mHOcVtsHLMuesJkswc0GZ)",
    "8. \"Barracuda\" by Heart (spotify_id: 4KfSdst7rW39C0sfhArdrz)",
    "9. \"Like a Stone\" by Audioslave (spotify_id: 3YuaBvuZqcwN3CEAyyoaei)",
    "10. \"Back In Black\" by AC/DC (spotify_id: 08mG3Y1vljYA6bvDt4Wqkj)"
]

spotify_ids = extract_spotify_ids(strings)
print(spotify_ids)


['6FRwDxXsvSasw0y2eDArsz', '5uvosCdMlFdTXhoazkTI5R', '6mHOcVtsHLMuesJkswc0GZ', '0sgDEFq9oeC8ueOK0mR5IA', '4KfSdst7rW39C0sfhArdrz', '5uvosCdMlFdTXhoazkTI5R', '4KfSdst7rW39C0sfhArdrz', '3YuaBvuZqcwN3CEAyyoaei', '08mG3Y1vljYA6bvDt4Wqkj']


In [189]:
import re

# Sample longer strings
longer_strings = [
    "1. \"Yesterday\" by The Beatles (page 17, row 1) - Spotify ID: 6FRwDxXsvSasw0y2eDArsz",
    'spotify\_id: "6FRwDxXsvSasw0y2eDArsz")',
    'rt (spotify\_id: 4KfSdst7rW39C0sfhArdrz)'
]

# Regex pattern
regex_pattern = r'\b[A-Za-z0-9]{22}\b'

# Extract alphanumeric strings of length 22 from longer strings
for string in longer_strings:
    matches = re.findall(regex_pattern, string)
    for match in matches:
        print(match)


6FRwDxXsvSasw0y2eDArsz
6FRwDxXsvSasw0y2eDArsz
4KfSdst7rW39C0sfhArdrz


<>:6: SyntaxWarning: invalid escape sequence '\_'
<>:7: SyntaxWarning: invalid escape sequence '\_'
<>:6: SyntaxWarning: invalid escape sequence '\_'
<>:7: SyntaxWarning: invalid escape sequence '\_'
/var/folders/1l/pycwvg157rv_sn0s68n1_y8c0000gn/T/ipykernel_84841/4172587299.py:6: SyntaxWarning: invalid escape sequence '\_'
  'spotify\_id: "6FRwDxXsvSasw0y2eDArsz")',
/var/folders/1l/pycwvg157rv_sn0s68n1_y8c0000gn/T/ipykernel_84841/4172587299.py:7: SyntaxWarning: invalid escape sequence '\_'
  'rt (spotify\_id: 4KfSdst7rW39C0sfhArdrz)'


In [166]:
import re

def extract_spotify_id(text):
    pattern = r"(?:Spotify ID|spotify_id)[:\s]*\"?([a-zA-Z0-9_-]{22})\"?"
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None


# Test the function with your examples
strings = [
    "\"Yesterday\" by The Beatles (page 17, row 1, spotify_id: \"6FRwDxXsvSasw0y2eDArsz\")",
    "\"Imagine\" by John Lennon (page 17, row 2, spotify_id: \"1bp2IO61zbQrbWNmKKxg3f\")",
    "\"Light My Fire\" by The Doors (spotify_id: 5uvosCdMlFdTXhoazkTI5R)",
    "\"The Logical Song - 2010 Remastered\" by Supertramp (spotify_id: 6mHOcVtsHLMuesJkswc0GZ)",
    "\"Barracuda\" by Heart (spotify_id: 4KfSdst7rW39C0sfhArdrz)",
    "\"Like a Stone\" by Audioslave (spotify_id: 3YuaBvuZqcwN3CEAyyoaei)",
    "\"Back In Black\" by AC/DC (spotify_id: 08mG3Y1vljYA6bvDt4Wqkj)",
    "\"Kinetic (The Crystal Method vs. Dada Life)\" by The Crystal Method, with a danceability score of 0.616. Spotify ID: 6s0yNmp4Hd32wGx40T6uL8",
    "\"Galvanize\" by The Chemical Brothers, with a danceability score of 0.745. Spotify ID: 4bz7uB4edifWKJXSDxwHcs",
    "\"God Is a DJ\" by Faithless, with a danceability score of 0.606. Spotify ID: 1pUFYb9peWkK8m1WCKNRjp",
    "\"In Yer Face - Bicep Remix\" by 808 State, with a danceability score of 0.616. Spotify ID: 6DxKG8EHEqkWdKrFQSvm32",
    "\"Teardrop\" by Massive Attack, with a danceability score of 0.615. Spotify ID: 67Hna13dNDkZvBpTXRIaOJ",
    "\"Hallelujah\" by Leonard Cohen (page 17, row 3, spotify_id: \"3qiyyUfYe7CRYLucrPmulD\")",
    "\"Someone Like You\" by Adele (page 18, row 1, spotify_id: \"5uvosCdMlFdTXhoazkTI5R\")",
    "\"Fix You\" by Coldplay (page 18, row 2, spotify_id: \"6mHOcVtsHLMuesJkswc0GZ\")"
]

for string in strings:
    print(extract_spotify_id(string))


6FRwDxXsvSasw0y2eDArsz
1bp2IO61zbQrbWNmKKxg3f
5uvosCdMlFdTXhoazkTI5R
6mHOcVtsHLMuesJkswc0GZ
4KfSdst7rW39C0sfhArdrz
3YuaBvuZqcwN3CEAyyoaei
08mG3Y1vljYA6bvDt4Wqkj
6s0yNmp4Hd32wGx40T6uL8
4bz7uB4edifWKJXSDxwHcs
1pUFYb9peWkK8m1WCKNRjp
6DxKG8EHEqkWdKrFQSvm32
67Hna13dNDkZvBpTXRIaOJ
3qiyyUfYe7CRYLucrPmulD
5uvosCdMlFdTXhoazkTI5R
6mHOcVtsHLMuesJkswc0GZ


ignore below

In [161]:
len("6DxKG8EHEqkWdKrFQSvm32")

22

In [ ]:
#convert to list of string



In [90]:
# Function to convert Spotify IDs to clickable links
def spotify_id_to_link(spotify_id):
    return f"https://open.spotify.com/track/{spotify_id}"

# Output the clickable links
for idx, spotify_id in enumerate(spotify_ids, start=1):
    link = spotify_id_to_link(spotify_id)
    print(f"{idx}. {link}")


1. https://open.spotify.com/track/s
2. https://open.spotify.com/track/p
3. https://open.spotify.com/track/o
4. https://open.spotify.com/track/t
5. https://open.spotify.com/track/i
6. https://open.spotify.com/track/f
7. https://open.spotify.com/track/y
8. https://open.spotify.com/track/_
9. https://open.spotify.com/track/i
10. https://open.spotify.com/track/d
11. https://open.spotify.com/track/s
